In [0]:
from google.colab import files

import pandas as pd
import numpy as np
import csv

In [0]:
!pip install annoy

     |████████████████████████████████| 645kB 2.8MB/s 
  Created wheel for annoy: filename=annoy-1.16.3-cp36-cp36m-linux_x86_64.whl size=297342 sha256=873538073a0b4451f78e7b29b9d6f20556e2741a384cbe504dda0e661251d42e
  Stored in directory: /root/.cache/pip/wheels/f3/01/54/6ef760fe9f9fc6ba8c19cebbe6358212b5f3b5b0195c0b813f
Successfully built annoy


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# @title Data processing: test_set and ruwordnet vectors

df_nouns = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fasttext vectors/nouns_private.csv')
df_ruwordnet = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fasttext vectors/ruWordNet_vectors.csv')

nouns_names = df_nouns['Word'].to_list()
nouns_vectors = df_nouns['Vector'].to_list()

ruwordnet_names = df_ruwordnet['Word'].to_list()
ruwordnet_vectors = df_ruwordnet['Vector'].to_list()

import ast

for i in range(len(nouns_vectors)):
    nouns_vectors[i] = ast.literal_eval(nouns_vectors[i])

for i in range(len(ruwordnet_vectors)):
    ruwordnet_vectors[i] = ast.literal_eval(ruwordnet_vectors[i])

nouns_dict = dict(zip(nouns_names, nouns_vectors))
ruwordnet_dict = dict(zip(ruwordnet_names, ruwordnet_vectors))

In [0]:
# @title Data processing: train dataset

df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fasttext vectors/training_nouns.tsv', sep='\t')

train_hyponyms = df_train['TEXT'].str.lower().to_list()
train_hypernyms = df_train['PARENT_TEXTS'].str.lower().to_list()

for i in range(len(train_hypernyms)):
    train_hypernyms[i] = ast.literal_eval(train_hypernyms[i])
    new_list = list()
    for j in range(len(train_hypernyms[i])):
        string_ = train_hypernyms[i][j]
        list_ = string_.split(',')
        for k in range(len(list_)):
            new_list.append(list_[k])
    train_hypernyms[i] = new_list

In [0]:
# @title FastText model for new words

from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('/content/drive/My Drive/Colab Notebooks/ft_native_300_ru_wiki_lenta_lower_case.bin')

In [0]:
# @title Build AnnoyIndex

from annoy import AnnoyIndex
import random

f = 300
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
for i in range(len(ruwordnet_names)):
    v = ruwordnet_vectors[i]
    t.add_item(i, v)

t.build(10) # (number) of trees

True

In [0]:
# @title Count true hypernymms in topN for train_set

count_true = np.zeros(len(train_hyponyms))

for i in range(len(train_hyponyms)):
    hyponim = train_hyponyms[i]
    hyponim = hyponim.split(',')[0]
    hypernyms = train_hypernyms[i]
    v = model[hyponim]
    idxs = t.get_nns_by_vector(v, 400)
    words = [ruwordnet_names[j] for j in idxs]
    for w in words:
        for h in hypernyms:
            if w == h:
                count_true[i] += 1

print(np.where(count_true>0)[0].shape[0])
print(count_true.shape[0])

1407
17242


In [0]:
# @title Save to csv

top100_dict = dict()

for noun in nouns_names:
    v = nouns_dict[noun]
    idxs = t.get_nns_by_vector(v, 200)
    words = [ruwordnet_names[i] for i in idxs]
    top100_dict[noun] = words

import csv

with open('top100.csv', 'w') as f:
    for key in top100_dict.keys():
        f.write("%s,%s\n"%(key,top100_dict[key]))